# Tensoren in der GPU

## Abfrage der GPU

Mit Hilfe des `cuda`-Moduls von `PyTorch` kann eine vorhandene GPU erkannt und abgefragt werden:

In [1]:
import torch

In [2]:
def check_gpu(device = None):
    if torch.cuda.device_count() == 0:
        print("Keine GPU gefunden, verwende CPU")
        return 'cpu'
    else:
        print(f"GPU gefunden: {torch.cuda.get_device_name(0)}")
        return 'cuda'

        
def gpu_info():
    print(f'CUDA available: {torch.cuda.is_available()}')
    print(f'CUDA devices count: {torch.cuda.device_count()}')
    print(f'CUDA current device: {torch.cuda.current_device()}')
    print(f'CUDA device 0: {torch.cuda.device(0)}')
    print(f'CUDA device name: {torch.cuda.get_device_name(0)}')

In [3]:
check_gpu()
gpu_info()

GPU gefunden: NVIDIA GeForce GTX 1050 Ti
CUDA available: True
CUDA devices count: 1
CUDA current device: 0
CUDA device 0: <torch.cuda.device object at 0x00000198F84B8D10>
CUDA device name: NVIDIA GeForce GTX 1050 Ti


Mit der folgenden Anweisung wird eine Variable `device` definiert, die bei der Erzeugung von Tensoren verwendet werden kann:

In [4]:
device =  "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

### Erzeugung von Tensoren in der GPU

In [5]:
x = torch.tensor([[1,2,3],[4,5,6]],device = device)
y = torch.tensor([[7,8,9],[10,11,12]],device = device)
x,y

(tensor([[1, 2, 3],
         [4, 5, 6]], device='cuda:0'),
 tensor([[ 7,  8,  9],
         [10, 11, 12]], device='cuda:0'))

Liegen die Ausgangs-Tensoren auf der GPU, so auch die Ergebnisse von Beechnungen:

In [6]:
z = x + y
z

tensor([[ 8, 10, 12],
        [14, 16, 18]], device='cuda:0')

## Performance

Auf der GPU werden alle Berechnungen erheblich schneller durchgeführt:

In [7]:
N = 20000


In [8]:
m1_gpu = torch.rand(N,N,dtype = float, device = 'cuda')
m2_gpu = torch.rand(N,N,dtype = float, device = 'cuda') 
%timeit -n 5 -r 1 m1_gpu @ m2_gpu

146 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [9]:
m1_cpu = torch.rand(N,N,dtype = float, device = 'cpu')
m2_cpu = torch.rand(N,N,dtype = float, device = 'cpu') 
#%timeit -n 5 -r 1 m1_gpu @ m2_gpu

In [12]:
import torch
import time

# Funktion zur Durchführung der Matrixmultiplikation und Zeitmessung
def benchmark(device, size=10000):
    # Erstelle zufällige Matrizen
    a = torch.randn(size, size, device=device)
    b = torch.randn(size, size, device=device)
    
    # Zeitmessung
    start_time = time.time()
    c = torch.mm(a, b)
    torch.cuda.synchronize() # Synchronisiere, um genaue Zeit zu erhalten
    end_time = time.time()
    
    return end_time - start_time

# Benchmark auf CPU
cpu_time = benchmark('cpu')
print(f"CPU Zeit: {cpu_time:.4f} Sekunden")

# Benchmark auf GPU, falls verfügbar
if torch.cuda.is_available():
    gpu_time = benchmark('cuda')
    print(f"GPU Zeit: {gpu_time:.4f} Sekunden")
else:
    print("Keine GPU verfügbar.")

CPU Zeit: 653.4897 Sekunden
GPU Zeit: 9.4593 Sekunden
